# Setup

In [1]:
!pip install pyspark

In [112]:
from pyspark.ml.classification import LinearSVC, GBTClassifier, LogisticRegression
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit, CrossValidator
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
from pyspark.ml.linalg import VectorUDT, Vectors
from pyspark.sql import types as t
import numpy as np

In [3]:
spark = (
    SparkSession.builder.config("spark.executor.memory", "4g")
    .config("spark.executor.cores", "2")
    .config("spark.cores.max", "2")
    .config("spark.driver.memory", "8g")
    .getOrCreate()
)

# Read the dataset(s)

Make sure to upload the train datasets generated by the `make-dataset.sh` script before proceeding.
Either train using X4 or X3,X2

In [ ]:
!unzip X4.zip
df = spark.read.parquet('./X4.parquet')

In [8]:
!unzip X2.zip
!unzip X3.zip
df1 = spark.read.parquet('./X2.parquet')
df2 = spark.read.parquet('./X3.parquet')
df = df1.unionAll(df2)
df.toPandas()

Archive:  X2.zip
replace X2.parquet/.part-00001-865d672c-b16b-4632-b255-74ba3c1c9779-c000.snappy.parquet.crc? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C
/bin/bash: y: command not found
Archive:  X3.zip
replace X3.parquet/part-00000-d48b5338-63f5-48aa-b777-5ec06012bc6b-c000.snappy.parquet? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C
/bin/bash: y: command not found


,features,label
0,"[0.32666666666666666, 1.0, 1.0, 1.0, 0.2407407...",1
1,"[0.5026178010471204, 1.0, 1.0, 0.0, 0.0, 0.0, ...",1
2,"[0.17842323651452285, 1.0, 1.0, 1.0, 0.3611111...",1
3,"[0.7969348659003832, 1.0, 1.0, 1.0, 1.0, 0.0, ...",1
4,"[0.4024896265560166, 1.0, 1.0, 1.0, 0.24074074...",1
...,...,...
230533,"[0.2574850299401198, 1.0, 1.0, 0.0, 0.21428571...",0
230534,"[0.2857142857142857, 1.0, 1.0, 0.0, 0.28205128...",0
230535,"[0.2544378698224852, 1.0, 0.0, 0.0, 0.07999999...",0
230536,"[0.2544378698224852, 1.0, 1.0, 0.0, 0.21428571...",0


In [104]:
def stratified_split_train_test(df, frac, label,seed=42):
    fractions = df.select(label).distinct().withColumn("fraction", f.lit(frac)).rdd.collectAsMap()
    df_frac = df.stat.sampleBy(label, fractions, seed)
    df_remaining = df.exceptAll(df_frac)
    return df_frac, df_remaining

train_set, test_set = stratified_split_train_test(df, 0.8, 'label')

def with_weights(df, column="label"):
    w_zero = 1 / df.filter(f.col(column) == 0).count()
    w_one = 0.8 / df.filter(f.col(column) == 1).count()
    return df.withColumn("weight", f.when(f.col(column) == 0, w_zero).otherwise(w_one))

train_set = with_weights(train_set)

# Train SVM

In [105]:
model_svc = LinearSVC(weightCol='weight', maxIter=100, predictionCol='prediction_svc')
paramGrid = ParamGridBuilder()\
    .addGrid(model_svc.threshold, [1.2,1.3,1.4,1.5,1.65,1.7])\
    .build()
cvs = CrossValidator(estimator=model_svc,
                     estimatorParamMaps=paramGrid,
                     evaluator=BinaryClassificationEvaluator(weightCol='weight',metricName='areaUnderPR'),
                     numFolds=4)
estimator_svc = cvs.fit(train_set_base)

# Prediction and accuracy

In [ ]:
prediction = estimator_svc.transform(test_set).select('label', 'prediction')

tp = prediction.filter('label == 1 AND prediction == 1').count()
rp = prediction.filter('prediction == 1').count()
ap = prediction.filter('label == 1').count()

p = tp/rp
r = tp/ap
f = 2 * p * r / (p + r)
print(f'F1-score: {f}')